In [1]:
%run CC.ipynb

In [2]:
OrderLog20020812_df = pd.read_csv('./resources/OrderLog20020812.txt')
OrderLog20020812_df.head()

,NO,SECCODE,BUYSELL,TIME,ORDERNO,ACTION,PRICE,VOLUME,TRADENO,TRADEPRICE
0,1,RU14VOST1005,S,103000000,1,1,2.380,4500,NaN,NaN
1,2,RU0009071187,S,103000000,2,1,990.000,300,NaN,NaN
2,3,RU14VOST1005,S,103000000,3,1,2.420,21600,NaN,NaN
3,4,RU0009100762,B,103000000,4,1,0.133,100000,NaN,NaN
4,5,RU0009054449,S,103000000,5,1,332.000,500,NaN,NaN


In [3]:
dic = {4434:0, 'asdf':1}
dic[4434], dic['asdf'], len(dic)

(0, 1, 2)

In [4]:
class taq2edges:
    def __init__(self, taq_df,
              row_id_col, # unique row id
              sec_col, # column with security code
              link_col # orderno or some mark linking trades and quotes
             ):
        self.taq_df = taq_df.copy()
        self.row_id_col = row_id_col
        self.sec_col = sec_col
        self.link_col = link_col
        self.secs = pd.unique(taq_df[sec_col])
        self.vertex_count_dic = {k:v for (k, v) in zip(self.secs, [0 for i in range(len(self.secs))])}
        self.rowid2gkey_dic = {k:v for (k, v) in zip(self.secs, [{} for i in range(len(self.secs))])}
        self.lnk2gkey_dic = {k:v for (k, v) in zip(self.secs, [{} for i in range(len(self.secs))])}
        self.edge_tpls = {k:v for (k, v) in zip(self.secs, [[] for i in range(len(self.secs))])
        
        self.rows_to_edges()
                          
        self.Gs = {k:v for (k, v) in zip(self.secs, [Graph(self.edge_tpls[sec],
                                                           self.vertex_count_dic[sec]) for sec in self.secs])}
        self.CCs = {CC(self.Gs[sec]) for sec in self.secs}

    def sec_rowid_lnk2edges(self, rowid, sec, lnk): 
        # give 0, 1, 2, ... vertex names to all raw ids and "links" (eg ordernos)
        if rowid not in self.rowid2gkey_dic[sec]:
            self.rowid2gkey_dic[sec][rowid] = self.vertex_count_dic[sec]
            self.vertex_count_dic[sec] += 1
        if lnk not in self.lnk2gkey_dic[sec]:
            self.lnk2gkey_dic[sec][lnk] = self.vertex_count_dic[sec]
            self.vertex_count_dic[sec] += 1
        self.edge_tpls[sec].append((self.rowid2gkey_dic[sec][rowid], 
                               self.lnk2gkey_dic[sec][lnk]))

    def rows_to_edges(self):
        [self.sec_rowid_lnk2edges(rowid, sec, lnk) 
         for rowid, sec, lnk in zip(self.taq_df[self.row_id_col],
                                    self.taq_df[self.sec_col],
                                    self.taq_df[self.link_col])]
        

In [5]:
temp = taq2edges(OrderLog20020812_df, 'NO', 'SECCODE', 'ORDERNO')

RU14VOST1005
(0, 1)
(2, 3)
(4, 5)
(6, 7)
(8, 9)
(10, 11)
(12, 13)
(14, 15)
(16, 17)
(18, 19)
(20, 21)
(22, 23)
(24, 25)
(26, 27)
(28, 29)
(30, 31)
(32, 33)
(34, 35)
(36, 37)
(38, 39)
(40, 41)
(42, 43)
(44, 45)
(46, 47)
(48, 49)
(50, 51)
(52, 53)
(54, 55)
(56, 57)
(58, 59)
(60, 61)
(62, 63)
(64, 65)
(66, 67)
(68, 69)
(70, 71)
(72, 73)
(74, 75)
(76, 77)
(78, 79)
(80, 81)
(82, 83)
(84, 57)
(85, 86)
(87, 88)
(89, 90)
(91, 92)
(93, 47)
(94, 95)
(96, 65)
(97, 98)
(99, 67)
(100, 101)
(102, 103)
(104, 105)
(106, 107)
(108, 109)
(110, 111)
(112, 113)
(114, 69)
(115, 116)
(117, 118)
(119, 120)
(121, 122)
(123, 71)
(124, 125)
(126, 127)
(128, 15)
(129, 130)
(131, 103)
(132, 133)
(134, 135)
(136, 137)
(138, 139)
(140, 141)
(142, 143)
(144, 145)
(146, 147)
(148, 149)
(150, 151)
(152, 153)
(154, 155)
(156, 157)
(158, 159)
(160, 161)
(162, 163)
(164, 165)
(166, 167)
(168, 75)
(169, 170)
(171, 172)
(173, 174)
(175, 176)
(177, 178)
(179, 180)
(181, 180)
(182, 98)
(183, 98)
(184, 185)
(186, 141)
(187, 1

(1426, 1406)
(1427, 1428)
(1429, 1430)
(1431, 1420)
(1432, 1430)
(1433, 1367)
(1434, 1435)
(1436, 1437)
(1438, 1439)
(1440, 1435)
(1441, 1442)
(1443, 1444)
(1445, 1422)
(1446, 1447)
(1448, 1442)
(1449, 1450)
(1451, 255)
(1452, 1453)
(1454, 1455)
(1456, 1457)
(1458, 1459)
(1460, 1450)
(1461, 1462)
(1463, 1464)
(1465, 1466)
(1467, 1468)
(1469, 1468)
(1470, 1462)
(1471, 1472)
(1473, 1474)
(1475, 1476)
(1477, 1476)
(1478, 1474)
(1479, 1480)
(1481, 1472)
(1482, 1459)
(1483, 1484)
(1485, 1457)
(1486, 1455)
(1487, 1488)
(1489, 1484)
(1490, 1491)
(1492, 1493)
(1494, 1493)
(1495, 1496)
(1497, 1496)
(1498, 1499)
(1500, 1499)
(1501, 1502)
(1503, 1504)
(1505, 1502)
(1506, 1504)
(1507, 1508)
(1509, 1510)
(1511, 1512)
(1513, 1514)
(1515, 1516)
(1517, 1516)
(1518, 1447)
(1519, 1016)
(1520, 1439)
(1521, 1522)
(1523, 1522)
(1524, 1525)
(1526, 1527)
(1528, 1502)
(1529, 1527)
(1530, 1018)
(1531, 1532)
(1533, 1453)
(1534, 1535)
(1536, 1535)
(1537, 1447)
(1538, 1539)
(1540, 1539)
(1541, 1437)
(1542, 1022)


(2204, 2205)
(2206, 2207)
(2208, 2187)
(2209, 2189)
(2210, 2211)
(2212, 2207)
(2213, 2179)
(2214, 2215)
(2216, 2217)
(2218, 2219)
(2220, 2133)
(2221, 2222)
(2223, 2192)
(2224, 2139)
(2225, 2226)
(2227, 2228)
(2229, 1546)
(2230, 1536)
(2231, 1528)
(2232, 1514)
(2233, 2197)
(2234, 1494)
(2235, 2236)
(2237, 2238)
(2239, 2240)
(2241, 2228)
(2242, 2243)
(2244, 2245)
(2246, 2243)
(2247, 2245)
(2248, 2236)
(2249, 2245)
(2250, 2226)
(2251, 2245)
(2252, 2199)
(2253, 2245)
(2254, 2172)
(2255, 2245)
(2256, 2205)
(2257, 2245)
(2258, 1068)
(2259, 2245)
(2260, 2092)
(2261, 2245)
(2262, 2101)
(2263, 2245)
(2264, 2238)
(2265, 2266)
(2267, 2268)
(2269, 2270)
(2271, 2272)
(2273, 2268)
(2274, 2275)
(2276, 2277)
(2278, 2279)
(2280, 2281)
(2282, 2281)
(2283, 2275)
(2284, 2281)
(2285, 2272)
(2286, 1448)
(2287, 2288)
(2289, 1397)
(2290, 2291)
(2292, 2293)
(2294, 2295)
(2296, 2297)
(2298, 2185)
(2299, 2300)
(2301, 2302)
(2303, 2222)
(2304, 2305)
(2306, 437)
(2307, 2308)
(2309, 2128)
(2310, 2311)
(2312, 2313)


(569, 570)
(571, 572)
(573, 574)
(575, 576)
(577, 558)
(578, 579)
(580, 581)
(582, 583)
(584, 585)
(586, 568)
(587, 532)
(588, 589)
(590, 591)
(592, 593)
(594, 595)
(596, 597)
(598, 599)
(600, 601)
(602, 603)
(604, 605)
(606, 591)
(607, 570)
(608, 603)
(609, 610)
(611, 612)
(613, 579)
(614, 615)
(616, 617)
(618, 383)
(619, 620)
(621, 595)
(622, 623)
(624, 555)
(625, 626)
(627, 628)
(629, 630)
(631, 632)
(633, 634)
(635, 636)
(637, 623)
(638, 639)
(640, 641)
(642, 643)
(644, 645)
(646, 610)
(647, 648)
(649, 650)
(651, 641)
(652, 650)
(653, 520)
(654, 655)
(656, 655)
(657, 650)
(658, 659)
(660, 661)
(662, 626)
(663, 661)
(664, 659)
(665, 661)
(666, 615)
(667, 661)
(668, 669)
(670, 669)
(671, 661)
(672, 526)
(673, 674)
(675, 676)
(677, 678)
(679, 458)
(680, 681)
(682, 683)
(684, 593)
(685, 643)
(686, 314)
(687, 688)
(689, 572)
(690, 688)
(691, 589)
(692, 688)
(693, 681)
(694, 688)
(695, 550)
(696, 688)
(697, 561)
(698, 688)
(699, 700)
(701, 702)
(703, 704)
(705, 702)
(706, 707)
(708, 709)

(4067, 4062)
(4068, 3896)
(4069, 4070)
(4071, 4072)
(4073, 4072)
(4074, 4075)
(4076, 4075)
(4077, 4056)
(4078, 3891)
(4079, 4080)
(4081, 4082)
(4083, 4084)
(4085, 4086)
(4087, 4086)
(4088, 4056)
(4089, 4090)
(4091, 4092)
(4093, 4094)
(4095, 4025)
(4096, 4097)
(4098, 4092)
(4099, 4082)
(4100, 4101)
(4102, 4103)
(4104, 4103)
(4105, 4097)
(4106, 4107)
(4108, 4109)
(4110, 4111)
(4112, 4094)
(4113, 4114)
(4115, 4116)
(4117, 4118)
(4119, 4120)
(4121, 4120)
(4122, 4097)
(4123, 4124)
(4125, 4124)
(4126, 4097)
(4127, 4124)
(4128, 4114)
(4129, 4111)
(4130, 4116)
(4131, 4132)
(4133, 4132)
(4134, 4114)
(4135, 4136)
(4137, 4132)
(4138, 4136)
(4139, 4140)
(4141, 2321)
(4142, 4143)
(4144, 3989)
(4145, 4146)
(4147, 4140)
(4148, 4146)
(4149, 4150)
(4151, 4140)
(4152, 4150)
(4153, 4154)
(4155, 3987)
(4156, 4157)
(4158, 4157)
(4159, 4150)
(4160, 4161)
(4162, 2639)
(4163, 4164)
(4165, 4164)
(4166, 4150)
(4167, 4168)
(4169, 4170)
(4171, 4172)
(4173, 4172)
(4174, 4107)
(4175, 4176)
(4177, 4178)
(4179, 4180)

...

(5489, 5472)
(5490, 5491)
(5492, 5493)
(5494, 5491)
(5495, 5496)
(5497, 5484)
(5498, 5499)
(5500, 5501)
(5502, 5468)
(5503, 5504)
(5505, 5506)
(5507, 5445)
(5508, 4787)
(5509, 5510)
(5511, 5512)
(5513, 5510)
(5514, 5512)
(5515, 5463)
(5516, 5517)
(5518, 5519)
(5520, 5521)
(5522, 5523)
(5524, 5521)
(5525, 5517)
(5526, 5487)
(5527, 25)
(5528, 5529)
(5530, 5529)
(5531, 5517)
(5532, 5533)
(5534, 5535)
(5536, 5352)
(5537, 5535)
(5538, 5539)
(5540, 5541)
(5542, 5519)
(5543, 5544)
(5545, 5407)
(5546, 5547)
(5548, 5493)
(5549, 5550)
(5551, 5552)
(5553, 5544)
(5554, 5555)
(5556, 5557)
(5558, 5466)
(5559, 5560)
(5561, 660)
(5562, 5563)
(5564, 5565)
(5566, 5421)
(5567, 5568)
(5569, 5517)
(5570, 5571)
(5572, 5352)
(5573, 5571)
(5574, 5555)
(5575, 5576)
(5577, 5547)
(5578, 5579)
(5580, 5315)
(5581, 5582)
(5583, 5584)
(5585, 5586)
(5587, 5588)
(5589, 5590)
(5591, 5590)
(5592, 5571)
(5593, 5523)
(5594, 5576)
(5595, 5596)
(5597, 5584)
(5598, 5596)
(5599, 5479)
(5600, 5601)
(5602, 5603)
(5604, 5504)
(5

(31, 32)
(33, 5)
(34, 21)
(35, 36)
(37, 38)
(39, 24)
(40, 41)
(42, 43)
(44, 45)
(46, 47)
(48, 38)
(49, 41)
(50, 17)
(51, 52)
(53, 54)
(55, 54)
(56, 57)
(58, 59)
(60, 61)
(62, 52)
(63, 64)
(65, 66)
(67, 66)
(68, 64)
(69, 70)
(71, 47)
(72, 73)
(74, 70)
(75, 76)
(77, 78)
(79, 36)
(80, 81)
(82, 57)
(83, 84)
(85, 45)
(86, 87)
(88, 89)
(90, 91)
(92, 73)
(93, 94)
(95, 76)
(96, 97)
(98, 87)
(99, 100)
(101, 102)
(103, 84)
(104, 105)
(106, 107)
(108, 100)
(109, 110)
(111, 112)
(113, 94)
(114, 105)
(115, 116)
(117, 118)
(119, 97)
(120, 121)
(122, 123)
(124, 91)
(125, 126)
(127, 128)
(129, 128)
(130, 126)
(131, 132)
(133, 134)
(135, 116)
(136, 137)
(138, 139)
(140, 121)
(141, 142)
(143, 142)
(144, 145)
(146, 147)
(148, 78)
(149, 118)
(150, 107)
(151, 132)
(152, 134)
(153, 137)
(154, 139)
(155, 147)
(156, 145)
(157, 110)
(158, 112)
(159, 123)
(160, 89)
(161, 102)
(162, 81)
(163, 61)
(164, 59)
(165, 28)
(166, 30)
(167, 32)
(168, 43)
(169, 26)
(170, 19)
(171, 15)
(172, 3)
(173, 1)
(174, 175)
RU000900

(1957, 1820)
(1958, 1959)
(1960, 1956)
(1961, 1962)
(1963, 1964)
(1965, 1943)
(1966, 1964)
(1967, 1962)
(1968, 1969)
(1970, 1971)
(1972, 1973)
(1974, 1975)
(1976, 1977)
(1978, 1979)
(1980, 1981)
(1982, 1983)
(1984, 1985)
(1986, 1987)
(1988, 1939)
(1989, 1990)
(1991, 1969)
(1992, 1971)
(1993, 1973)
(1994, 1995)
(1996, 1990)
(1997, 1995)
(1998, 1999)
(2000, 2001)
(2002, 2003)
(2004, 2005)
(2006, 2005)
(2007, 1458)
(2008, 1999)
(2009, 2010)
(2011, 2003)
(2012, 2013)
(2014, 2015)
(2016, 2015)
(2017, 2018)
(2019, 2013)
(2020, 2021)
(2022, 2018)
(2023, 2024)
(2025, 2021)
(2026, 2027)
(2028, 2029)
(2030, 2024)
(2031, 2032)
(2033, 2034)
(2035, 2036)
(2037, 2036)
(2038, 1458)
(2039, 2040)
(2041, 2032)
(2042, 2043)
(2044, 2029)
(2045, 2046)
(2047, 2043)
(2048, 2040)
(2049, 2050)
(2051, 2052)
(2053, 2054)
(2055, 2050)
(2056, 2057)
(2058, 1959)
(2059, 2060)
(2061, 2057)
(2062, 1908)
(2063, 2064)
(2065, 1987)
(2066, 1985)
(2067, 1983)
(2068, 2054)
(2069, 1981)
(2070, 2071)
(2072, 2046)
(2073, 1979)

(180, 117)
(181, 182)
(183, 175)
(184, 185)
(186, 187)
(188, 187)
(189, 134)
(190, 17)
(191, 192)
(193, 194)
(195, 194)
(196, 197)
(198, 197)
(199, 200)
(201, 202)
(203, 202)
(204, 200)
(205, 206)
(207, 182)
(208, 136)
(209, 210)
(211, 206)
(212, 167)
(213, 214)
(215, 19)
(216, 217)
(218, 185)
(219, 220)
(221, 192)
(222, 220)
(223, 217)
(224, 214)
(225, 210)
(226, 178)
(227, 164)
(228, 152)
(229, 148)
(230, 138)
(231, 31)
(232, 80)
(233, 123)
(234, 125)
(235, 131)
(236, 129)
(237, 127)
(238, 119)
(239, 108)
(240, 103)
(241, 39)
(242, 55)
(243, 61)
(244, 63)
(245, 53)
(246, 51)
(247, 41)
(248, 1)
(249, 27)
(250, 23)
(251, 15)
(252, 11)
(253, 5)
(254, 3)
RU0009062285
(0, 1)
(2, 3)
(4, 5)
(6, 7)
(8, 9)
(10, 11)
(12, 13)
(14, 15)
(16, 17)
(18, 19)
(20, 21)
(22, 23)
(24, 25)
(26, 27)
(28, 29)
(30, 31)
(32, 33)
(34, 35)
(36, 37)
(38, 11)
(39, 40)
(41, 40)
(42, 43)
(44, 45)
(46, 15)
(47, 48)
(49, 50)
(51, 52)
(53, 54)
(55, 54)
(56, 57)
(58, 59)
(60, 61)
(62, 63)
(64, 65)
(66, 67)
(68, 69)
(70

(29, 12)
(30, 31)
(32, 33)
(34, 35)
(36, 37)
(38, 37)
(39, 35)
(40, 33)
(41, 31)
(42, 19)
(43, 28)
(44, 16)
(45, 14)
(46, 1)
(47, 3)
RU0007782421
(0, 1)
(2, 3)
(4, 5)
(6, 7)
(8, 9)
(10, 5)
(11, 12)
(13, 14)
(15, 16)
(17, 9)
(18, 19)
(20, 7)
(21, 22)
(23, 24)
(25, 24)
(26, 27)
(28, 29)
(30, 31)
(32, 33)
(34, 12)
(35, 31)
(36, 37)
(38, 37)
(39, 40)
(41, 16)
(42, 14)
(43, 44)
(45, 29)
(46, 47)
(48, 49)
(50, 44)
(51, 52)
(53, 40)
(54, 47)
(55, 56)
(57, 58)
(59, 60)
(61, 60)
(62, 49)
(63, 60)
(64, 33)
(65, 60)
(66, 27)
(67, 60)
(68, 58)
(69, 70)
(71, 72)
(73, 74)
(75, 76)
(77, 58)
(78, 79)
(80, 76)
(81, 82)
(83, 84)
(85, 82)
(86, 87)
(88, 79)
(89, 90)
(91, 87)
(92, 93)
(94, 84)
(95, 96)
(97, 98)
(99, 96)
(100, 101)
(102, 103)
(104, 22)
(105, 103)
(106, 107)
(108, 101)
(109, 110)
(111, 110)
(112, 93)
(113, 90)
(114, 72)
(115, 70)
(116, 74)
(117, 98)
(118, 107)
(119, 56)
(120, 52)
(121, 19)
(122, 3)
(123, 1)
RU0008538145
(0, 1)
(2, 3)
(4, 5)
(6, 7)
(8, 9)
(10, 11)
(12, 13)
(14, 9)
(15, 13)
(1